**Project to predict diabetes_mellitus**

Load the data

In [1]:
import pandas as pd
pd.set_option('display.max_columns',300)
df=pd.read_csv('../input/train-106/train.csv',index_col=0,header=0)
df.head()

,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,encounter_id,diabetes_mellitus
200,62.0,19.108088,0.0,0,1,157.5,4,1,99.0,0,5,0.547917,0.0,47.40,113.0,501.05,0.0,0.0,28.000000,4.880000,1.0,4.0,0.0,2.0,275.0,108.0,23.1,0.0,42.0,38.0,131.000000,36.7,1.0,24.6,71.0,24.0,71.0,24.0,90.0,58.0,96.0,31.0,96.0,31.0,37.0,8.0,100.0,84.0,180.0,48.0,180.0,48.0,37.2,36.5,41.0,24.0,41.0,24.0,90.0,70.0,63.000000,32.000000,63.000000,32.000000,37.0,11.0,100.0,97.0,104.0,53.0,104.0,53.0,36.7,36.7,28.000000,27.000000,6.500000,6.100000,4.880000,4.670000,275.0,53.0,28.000000,26.000000,7.9,7.2,25.7,23.1,205.0,205.0,3.5,3.2,137.000000,131.000000,24.6,24.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196304,1
201,17.0,21.484375,0.0,2,0,160.0,3,0,98.0,0,5,0.001389,0.0,55.00,122.0,703.03,0.0,0.0,4.000000,0.580000,3.0,6.0,0.0,4.0,91.0,164.0,31.3,1.0,52.0,7.0,138.000000,36.2,1.0,8.1,84.0,42.0,84.0,42.0,164.0,114.0,100.0,59.0,100.0,59.0,21.0,16.0,100.0,96.0,141.0,95.0,141.0,95.0,38.0,35.6,84.0,54.0,84.0,54.0,46.0,144.0,100.000000,74.000000,100.000000,74.000000,21.0,16.0,100.0,100.0,138.0,105.0,138.0,105.0,36.4,35.6,4.000000,3.000000,7.900000,7.500000,0.720000,0.580000,112.0,91.0,25.000000,23.000000,10.1,10.1,31.3,31.3,233.0,233.0,4.0,3.7,143.000000,138.000000,8.1,8.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,210042,0
202,73.0,22.851562,0.0,0,0,160.0,3,0,100.0,0,6,0.002778,0.0,58.50,121.0,401.01,0.0,0.0,17.000000,0.730000,4.0,6.0,0.0,5.0,178.0,154.0,39.7,1.0,130.0,55.0,138.000000,36.3,1.0,14.5,139.0,41.0,139.0,41.0,122.0,44.0,130.0,60.0,130.0,60.0,50.0,8.0,100.0,91.0,232.0,111.0,232.0,111.0,37.2,36.3,139.0,64.0,139.0,64.0,76.0,44.0,134.768877,84.586327,135.232672,84.688373,41.0,41.0,99.0,91.0,223.0,150.0,223.0,150.0,36.8,36.8,17.000000,15.000000,8.700000,8.500000,0.830000,0.730000,189.0,133.0,27.000000,27.000000,13.6,13.6,39.7,39.7,230.0,230.0,3.6,2.7,150.000000,138.000000,14.5,14.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,215183,1
203,70.0,27.037924,1.0,2,0,165.1,8,2,92.0,0,2,0.293750,0.0,73.70,308.0,1207.01,1.0,0.0,25.567613,1.466696,4.0,6.0,0.0,5.0,86.0,54.0,22.9,1.0,44.0,29.0,138.874501,35.5,1.0,19.4,60.0,42.0,60.0,42.0,78.0,54.0,65.0,56.0,65.0,56.0,29.0,12.0,100.0,95.0,108.0,87.0,108.0,87.0,37.0,35.5,60.0,60.0,60.0,60.0,64.0,54.0,66.000000,52.000000,66.207085,52.606137,12.0,12.0,100.0,99.0,101.0,101.0,101.0,101.0,35.9,35.9,25.584964,21.643764,8.126831,7.525662,1.510359,1.239426,161.0,86.0,24.046434,21.325195,9.0,8.2,30.0,22.9,142.0,99.0,4.4,3.4,141.463857,138.084375,19.4,9.3,0.0,0.0,0.0,0.0

In [2]:
df.shape

(129957, 105)

In [3]:
X=df.drop(columns='diabetes_mellitus')
y=df['diabetes_mellitus']

Check if Dataset is balanced or not

In [4]:
y.value_counts()

0    101853
1     28104
Name: diabetes_mellitus, dtype: int64

In [5]:
import xgboost as xgb

xgb_cf2=xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=5000, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
# train model

xgb_cf2.fit(X_train,y_train)

# get predicted probabilities

y_pred2 = xgb_cf2.predict_proba(X_test)[:,1]



/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:40:31] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
# create true and positive rates

false_positive_rate2, true_positive_rate2, threshold2 = roc_curve(y_test, y_pred2)

# plot ROC curve
plt.title("Reciever Operating Characteristic")

plt.plot(false_positive_rate2, true_positive_rate2, ls="-.",)
plt.plot([0, 0], [1,0], c=".7"), plt.plot([1, 1], c=".7")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()

In [ ]:

print("ROC AUC Score for model 2: {:.4f}".format(roc_auc_score(y_test,y_pred2)))

In [8]:
y_pred2 = xgb_cf2.predict(X_test)
n_correct2 = sum(y_pred2 == y_test)
print("Model 2 Accuracy {0:.4f}".format(n_correct2 / len(y_pred2)))

Model 2 Accuracy 0.8369


In [17]:
y_pred2


array([0, 0, 0, ..., 0, 0, 1])

In [9]:
test = pd.read_csv('../input/test-106/test.csv')
print(test.shape)

(10234, 105)


In [10]:
for col in test:
    print(col)

Unnamed: 0
age
bmi
elective_surgery
ethnicity
gender
height
hospital_admit_source
icu_admit_source
icu_id
icu_stay_type
icu_type
pre_icu_los_days
readmission_status
weight
apache_2_diagnosis
apache_3j_diagnosis
apache_post_operative
arf_apache
bun_apache
creatinine_apache
gcs_eyes_apache
gcs_motor_apache
gcs_unable_apache
gcs_verbal_apache
glucose_apache
heart_rate_apache
hematocrit_apache
intubated_apache
map_apache
resprate_apache
sodium_apache
temp_apache
ventilated_apache
wbc_apache
d1_diasbp_max
d1_diasbp_min
d1_diasbp_noninvasive_max
d1_diasbp_noninvasive_min
d1_heartrate_max
d1_heartrate_min
d1_mbp_max
d1_mbp_min
d1_mbp_noninvasive_max
d1_mbp_noninvasive_min
d1_resprate_max
d1_resprate_min
d1_spo2_max
d1_spo2_min
d1_sysbp_max
d1_sysbp_min
d1_sysbp_noninvasive_max
d1_sysbp_noninvasive_min
d1_temp_max
d1_temp_min
h1_diasbp_max
h1_diasbp_min
h1_diasbp_noninvasive_max
h1_diasbp_noninvasive_min
h1_heartrate_max
h1_heartrate_min
h1_mbp_max
h1_mbp_min
h1_mbp_noninvasive_max
h1_mbp_noni

In [20]:
X.shape
for col in X:
    print(col)


age
bmi
elective_surgery
ethnicity
gender
height
hospital_admit_source
icu_admit_source
icu_id
icu_stay_type
icu_type
pre_icu_los_days
readmission_status
weight
apache_2_diagnosis
apache_3j_diagnosis
apache_post_operative
arf_apache
bun_apache
creatinine_apache
gcs_eyes_apache
gcs_motor_apache
gcs_unable_apache
gcs_verbal_apache
glucose_apache
heart_rate_apache
hematocrit_apache
intubated_apache
map_apache
resprate_apache
sodium_apache
temp_apache
ventilated_apache
wbc_apache
d1_diasbp_max
d1_diasbp_min
d1_diasbp_noninvasive_max
d1_diasbp_noninvasive_min
d1_heartrate_max
d1_heartrate_min
d1_mbp_max
d1_mbp_min
d1_mbp_noninvasive_max
d1_mbp_noninvasive_min
d1_resprate_max
d1_resprate_min
d1_spo2_max
d1_spo2_min
d1_sysbp_max
d1_sysbp_min
d1_sysbp_noninvasive_max
d1_sysbp_noninvasive_min
d1_temp_max
d1_temp_min
h1_diasbp_max
h1_diasbp_min
h1_diasbp_noninvasive_max
h1_diasbp_noninvasive_min
h1_heartrate_max
h1_heartrate_min
h1_mbp_max
h1_mbp_min
h1_mbp_noninvasive_max
h1_mbp_noninvasive_min

In [22]:
X_test=test.drop(columns='Unnamed: 0')


In [23]:
X_test.head()

,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,encounter_id
0,72.0,29.549985,0.0,2,0,152.4,3,0,82.0,0,5,0.015278,0.0,67.706729,110.0,104.01,0.0,0.0,44.000000,1.490000,4.0,6.0,0.0,5.0,97.000000,38.0,39.900000,0.0,54.0,31.0,130.000000,36.4,0.0,5.400000,104.0,40.0,104.0,40.0,66.0,38.0,123.0,54.0,123.0,54.0,35.0,15.0,100.0,96.0,149.0,76.0,149.0,76.0,38.7,35.6,59.000000,59.000000,59.000000,59.000000,46.0,44.0,80.000000,80.000000,80.000000,80.000000,19.0,19.0,97.0,96.0,116.000000,116.000000,116.000000,116.000000,36.4,36.4,44.0,40.0,9.8,9.3,1.49,1.44,104.0,97.0,26.0,23.0,14.500000,14.200000,42.800000,39.90000,173.000000,173.000000,5.7,4.9,132.0,130.0,5.600000,5.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,144740
1,86.0,27.884391,0.0,2,0,175.3,2,0,82.0,0,5,0.000000,0.0,87.131254,117.0,106.01,0.0,0.0,19.000000,0.920000,4.0,6.0,0.0,5.0,73.000000,116.0,33.501676,0.0,41.0,53.0,142.000000,36.3,0.0,11.108199,101.0,27.0,101.0,27.0,116.0,56.0,129.0,41.0,129.0,41.0,51.0,14.0,100.0,95.0,181.0,65.0,181.0,65.0,36.8,36.4,85.000000,65.000000,85.000000,65.000000,116.0,104.0,129.000000,94.000000,129.000000,94.000000,31.0,24.0,97.0,95.0,181.000000,119.000000,181.000000,119.000000,36.7,36.7,19.0,19.0,8.5,8.5,1.00,1.00,102.0,73.0,27.0,27.0,11.304827,10.747775,34.957133,33.47874,209.201391,201.871382,3.9,3.8,144.0,144.0,11.438974,10.592914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141990
2,72.0,28.872265,0.0,2,0,162.6,3,1,82.0,0,5,0.003472,0.0,76.318442,302.0,109.01,0.0,0.0,24.000000,0.770000,4.0,6.0,0.0,5.0,84.000000,104.0,34.800000,0.0,117.0,38.0,137.000000,36.4,0.0,4.600000,97.0,61.0,97.0,61.0,98.0,68.0,117.0,77.0,117.0,77.0,49.0,8.0,99.0,85.0,146.0,103.0,146.0,103.0,36.9,35.9,97.000000,97.000000,97.000000,97.000000,98.0,92.0,117.000000,117.000000,117.000000,117.000000,20.0,20.0,99.0,96.0,146.000000,146.000000,146.000000,146.000000,36.4,36.4,24.0,16.0,9.1,8.5,0.82,0.77,141.0,84.0,32.0,31.0,11.800000,11.500000,36.600000,34.80000,253.000000,243.000000,3.6,3.6,141.0,137.0,6.600000,4.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,142038
3,66.0,29.267552,0.0,2,1,177.8,3,1,82.0,0,5,0.884028,0.0,92.182516,113.0,501.05,0.0,0.0,27.307062,1.559252,4.0,6.0,0.0,4.0,65.722134,110.0,34.035984,0.0,61.0,56.0,135.799448,36.6,1.0,25.249619,77.0,51.0,77.0,51.0,110.0,86.0,110.0,61.0,110.0,61.0,56.0,19.0,99.0,80.0,145.0,95.0,145.0,95.0,37.8,36.6,73.541276,60.640364,73.374467,60.617681,107.0,95.0,95.465677,78.627444,95.473883,78.730481,56.0,28.0,95.0,90.0,133.415517,112.254988,133.437999,112.323507,3

XGBoostError: [08:08:46] ../src/learner.cc:1183: Check failed: learner_model_param_.num_feature >= p_fmat->Info().num_col_ (104 vs. 105) : Number of columns does not match number of features in booster.
Stack trace:
  [bt] (0) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x8d264) [0x7f7e4c6e1264]
  [bt] (1) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x19c57a) [0x7f7e4c7f057a]
  [bt] (2) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x1a2639) [0x7f7e4c7f6639]
  [bt] (3) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(XGBoosterPredict+0xd5) [0x7f7e4c6d4b55]
  [bt] (4) /opt/conda/lib/python3.7/lib-dynload/../../libffi.so.7(+0x69dd) [0x7f7eccbe49dd]
  [bt] (5) /opt/conda/lib/python3.7/lib-dynload/../../libffi.so.7(+0x6067) [0x7f7eccbe4067]
  [bt] (6) /opt/conda/lib/python3.7/lib-dynload/_ctypes.cpython-37m-x86_64-linux-gnu.so(_ctypes_callproc+0x2f4) [0x7f7eccbfa944]
  [bt] (7) /opt/conda/lib/python3.7/lib-dynl

In [24]:
y_test = xgb_cf2.predict_proba(X_test)[:,1]

In [25]:
y_test

array([0.04386693, 0.1339118 , 0.26911697, ..., 0.10127831, 0.02759002,
       0.00728387], dtype=float32)

In [26]:
TARGET_COL = "diabetes_mellitus"

In [ ]:
test[TARGET_COL] = xgb_cf2.predict_proba(X_test)[:,1]

In [ ]:
test[["encounter_id","diabetes_mellitus"]].to_csv("submission.csv",index=False)